In [1]:
from langchain_openai import OpenAI,ChatOpenAI

llm=ChatOpenAI(
  model="qwen2.5-coder-1.5b-instruct",
  openai_api_key="EMPTY",
  base_url="http://127.0.0.1:1234/v1",
  temperature=0.7
)

In [2]:
from langchain.agents import tool

@tool
def get_word_length(word:str)->int:
    """Return the length of a word."""
    return len(word)
  
get_word_length.invoke("hello")

5

In [3]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()

@tool
def get_weather(location):
    """根据城市获取天气数据"""
    api_key = os.getenv("WEATHER_API_KEY")
    url = f"https://api.seniverse.com/v3/weather/now.json?key={api_key}&location={location}&language=zh-Hans&unit=c"
    response=requests.get(url)
    print(location)
    if response.status_code==200:
        data=response.json()
        weather={
            'description':data['results'][0]['now']['text'],
            'temperature':data['results'][0]['now']['temperature']
        }
        return weather
    else:
        raise Exception(f"失败接收天气信息：{response.status_code}")

In [4]:
get_weather.invoke("北京")

北京


{'description': '晴', 'temperature': '4'}

In [5]:
import json

@tool
def searxng_search(query: str):
    """输入搜索内容，使用 SearXNG 进行搜索。"""
    SEARXNG_URL = 'http://127.0.0.1:6688'
    SEARXNG_ENGINE_TOKEN = '123456789'
    
    params = {}
    # 设置搜索参数
    params['q'] = query
    params['format'] = 'json'  # 返回 JSON 格式的结果
    params['token'] = SEARXNG_ENGINE_TOKEN  # 加入引擎令牌
    
    # 发送 GET 请求
    response = requests.get(SEARXNG_URL, params=params)
    
    # 检查响应状态码
    if response.status_code == 200:
        res = response.json()
        resList = []
        for item in res['results']:
            resList.append({
                "title": item['title'],
                "content": item['content'],
                "url": item['url']
            })
            if len(resList)>=3:
                break
        return resList
    else:
        response.raise_for_status()
        
searxng_search.invoke("蔡徐坤")

[{'title': '蔡徐坤- 维基百科，自由的百科全书',
  'content': '蔡徐坤（1998年8月2日—），中国大陆歌手、原创音乐制作人及演员。2012年通过参加综艺节目《向上吧！少年》进入中国内地娱乐圈。2015年参加中韩真人秀节目《星动亚洲》第一季和\xa0...',
  'url': 'https://zh.wikipedia.org/zh-cn/%E8%94%A1%E5%BE%90%E5%9D%A4'},
 {'title': '蔡徐坤_百度百科',
  'content': '蔡徐坤（KUN），男，汉族，1998年8月2日出生于浙江省温州市，户籍湖南省吉首市，中国内地男歌手、演员、原创音乐制作人、MV导演。2012年8月，参演的偶像剧《童话二分之一》播出\xa0...',
  'url': 'https://baike.baidu.com/item/%E8%94%A1%E5%BE%90%E5%9D%A4/8511458'},
 {'title': '蔡徐坤',
  'content': '2025年11月14日 ... 在Apple Music 中畅听蔡徐坤的音乐。查找蔡徐坤的热门歌曲和专辑，包括《Deadman》和《What a Day》等作品。',
  'url': 'https://music.apple.com/cn/artist/%E8%94%A1%E5%BE%90%E5%9D%A4/1421021751'}]

In [6]:
from langchain_community.utilities import SearxSearchWrapper

search=SearxSearchWrapper(
    searx_host="http://127.0.0.1:6688",
    k=5,
)
search.run("蔡徐坤")

'4M followers · 53 following · 303 posts · @caixukun: “”\n\nShare your videos with friends, family, and the world.\n\nCai Xukun (born August 2, 1998), also known by the mononym Kun (stylized as KUN), is a Chinese singer-songwriter and actor. After participating in the first\xa0...\n\nAug 7, 2025 ... KUN 蔡徐坤•1.1M views · 25:41. Go to channel Spotlight by Laugh Society ... KUN丨Cai Xukun丨蔡徐坤 @KUNofficially Spotlight.\n\nMay 27, 2025 ... ... style is too powerful. 4:26. Go to channel KUN 蔡徐坤 · KUN - Jasmine (Official Video). KUN 蔡徐坤•1.6M views · 20:03. Go to channel Weird\xa0...'

In [7]:
from langchain_community.utilities import SerpAPIWrapper
search=SerpAPIWrapper()

from langchain.agents import Tool

searchTool=Tool(
    name="Search",
    func=search.run,
    description="useful for when you want to answer questions about current events"
)

In [8]:
tools=[get_word_length,get_weather,searchTool]

In [9]:
promptTemplate = """尽可能的帮助用户回答任何问题。

您可以使用以下工具来帮忙解决问题，如果已经知道了答案，也可以直接回答：

{tools}

回复格式说明
----------------------------

回复我时，请以下列两种格式之一输出回复：

选项 1：如果您希望人类使用工具，请使用此选项。
采用以下JSON模式格式化的回复内容：

```json
{{
    "reason": string, \\ 叙述使用工具的原因
    "action": string, \\ 要使用的工具。必须是 {tool_names} 之一
    "action_input": string \\ 工具的输入
}}
````

选项2：如果您认为您已经有答案或者已经通过使用工具找到了答案，想直接对人类做出反应，请使用此选项。采用以下JSON模式格式化的回复内容：

```json
{{
  "action": "Final Answer",
  "answer": string \\ 最终答复问题的答案放到这里！
}}
````
用户的输入 -------------------- 
这是用户的输入（请记住通过单个选项，以JSON模式格式化的回复内容，不要回复其他内容）：

{input}
"""

In [10]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt=ChatPromptTemplate.from_messages(
    [
      (
          "system",
          "你是一个非常有帮助的助手，你的任务是回答用户的问题。"
      ),
      (
          "human",
          promptTemplate
      ),
      MessagesPlaceholder(variable_name="agent_scratched")
    ]
)

In [11]:
from langchain.tools.render import render_text_description

prompt=prompt.partial(
    tools=render_text_description(list(tools)),
    tool_names=", ".join([t.name for t in tools])
)
prompt

ChatPromptTemplate(input_variables=['agent_scratched', 'input'], input_types={'agent_scratched': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annot

In [12]:
render_text_description(list(tools))

#get_word_length(word: str) -> int - Return the length of a word.\n
#get_weather(location) - 根据城市获取天气数据\n
#Search(query: str, **kwargs: Any) -> str - useful for when you want to answer questions about current events

'get_word_length(word: str) -> int - Return the length of a word.\nget_weather(location) - 根据城市获取天气数据\nSearch(query: str, **kwargs: Any) -> str - useful for when you want to answer questions about current events'

将工具绑定到LLM

创建代理

In [13]:
from langchain.agents.json_chat.prompt import TEMPLATE_TOOL_RESPONSE

TEMPLATE_TOOL_RESPONSE

"TOOL RESPONSE: \n---------------------\n{observation}\n\nUSER'S INPUT\n--------------------\n\nOkay, so what is the response to my last comment? If using information obtained from the tools you must mention it explicitly without mentioning the tool names - I have forgotten all TOOL RESPONSES! Remember to respond with a markdown code snippet of a json blob with a single action, and NOTHING else - even if you just want to respond to the user. Do NOT respond with anything except a JSON snippet no matter what!"

In [14]:
TEMPLATE_TOOL_RESPONSE="""
工具响应:
--------------------
{observation}

用户的输入:
--------------------
请根据工具的响应判断，是否能够回答问题:

{input}

请根据工具响应的内容，思考接下来回复。回复格式严格按照前面所说的2种JSON回复格式，选择其中1种进行回复。请记住通过单个选项，以JSON模式格式化的回复内容，不要回复其他内容。
"""

In [15]:
from langchain_core.messages import AIMessage,BaseMessage,HumanMessage

def format_log_to_messages(
    query,
    intermediate_steps,
    template_tool_response,
):
    """Construct the scratchpad that lets the agent continue its thought process."""
    thoughts:List[BaseMessage]=[]
    for action,observation in intermediate_steps:
        thoughts.append(AIMessage(content=action.log))
        human_message=HumanMessage(
           content=template_tool_response.format(
                observation=observation,
                input=query,
            )
        )
        thoughts.append(human_message)
    return thoughts

In [16]:
from langchain.agents.agent import AgentOutputParser
from langchain_core.utils.json import parse_json_markdown
from langchain_core.exceptions import OutputParserException
from langchain_core.agents import AgentAction,AgentFinish

class JSONAgentOutputParser(AgentOutputParser):
    """Parses tool invocations and final answers in JSON format.

    Expects output to be in one of two formats.

    If the output signals that an action should be taken, 
    should be in the below format. This will result in an AgentAction 
    being returned.

    ```
    {
      "action": "search",
      "action_input": "2+2"
    }
    ```
    If the output signals that a final answer should be given,
    should be in the below format. This will result in an AgentFinish 
    being returned.
    ```
    {
    "action": "Final Answer",
    "answer": "4"
    }
    ```
    """
    
    def parse(self,text):
        try:
            response=parse_json_markdown(text)
            if isinstance(response,list):
                logger.warning("Got multiple action response:%s",response)
                response=response[0]
            if response["action"]=="Final Answer":
                return AgentFinish({"answer":response["answer"]},text)
            else:
                return AgentAction(
                    response["action"],response.get("action_input",{}),text
                )
        except Exception as e:
            raise OutputParserException(f"Could not parse LLM output: {text}") from e
    
    @property
    def _type(self)->str:
        return "json-agent"

In [17]:
from langchain_core.runnables import Runnable,RunnablePassthrough

agent=(
    RunnablePassthrough.assign(
        agent_scratched=lambda x: format_log_to_messages(
            x["input"],
            x["intermediate_steps"],
            template_tool_response=TEMPLATE_TOOL_RESPONSE
        )
    )
    | prompt
    | llm
    | JSONAgentOutputParser()
)

In [18]:
from langchain.agents import AgentExecutor

agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True,return_intermediate_steps=True)

In [19]:
agent_executor.invoke({"input":"杭州的天气如何"})



> Entering new AgentExecutor chain...


```json
{
    "reason": "To get the weather of Hangzhou",
    "action": "get_weather",
    "action_input": "Hangzhou"
}
```Hangzhou
{'description': '晴', 'temperature': '18'}```json
{
    "action": "Final Answer",
    "answer": "杭州今天的天气晴朗，温度为18摄氏度。"
}
```

> Finished chain.


{'input': '杭州的天气如何',
 'answer': '杭州今天的天气晴朗，温度为18摄氏度。',
 'intermediate_steps': [(AgentAction(tool='get_weather', tool_input='Hangzhou', log='```json\n{\n    "reason": "To get the weather of Hangzhou",\n    "action": "get_weather",\n    "action_input": "Hangzhou"\n}\n```'),
   {'description': '晴', 'temperature': '18'})]}

In [20]:
agent_executor.invoke({"input":"刘德华的老婆是谁"})



> Entering new AgentExecutor chain...


以下是您的请求的解决方案：

```json
{
    "reason": "用户直接询问了刘德华的老婆是谁，因此需要通过搜索来获取答案。",
    "action": "Search",
    "action_input": "刘德华"
}
```

最终答复问题的答案放到这里！['Andy Lau Tak-wah SBS BBS MH JP, is a Hong Kong actor, singer-songwriter, film producer, and philanthropist. He was named the "Fourth Tiger" among the Five Tiger Generals of TVB in the 1980s as well as one of the Four Heavenly Kings in the 1990s.', 'Andy Lau (刘德华) type: Hong Kong actor and singer-songwriter.', 'Andy Lau (刘德华) entity_type: people.', 'Andy Lau (刘德华) kgmid: /m/01t2h2.', 'Andy Lau (刘德华) born: September 27, 1961 (age 64 years), Tai Hang, Hong Kong.', 'Andy Lau (刘德华) spouse: Carol Chu (m. 2008).', 'Andy Lau (刘德华) upcoming_movie: The Wandering Earth 3.', 'Andy Lau (刘德华) parents: Lau Lai.', 'Andy Lau (刘德华) height: 5′ 9″.', '刘德华，SBS，MH，JP（英语：Andy Lau Tak Wah；1961年9月27日—），男，汉族，籍贯广东新会，香港男演员、歌手、填词人、监制及出品人，1990年代获封为香港乐坛“四大天王”之一，也是 ...', '刘德华（Andy Lau），1961年9月27日出生于中国香港，华语影视男演员、流行乐歌手、制片人、作词人。1981年，出演个人首部电影《彩云曲》。1985年，发行个人首张音乐专辑《只 ...', '#刘德

{'input': '刘德华的老婆是谁',
 'answer': '刘德华的老婆是Carol Chu。',
 'intermediate_steps': [(AgentAction(tool='Search', tool_input='刘德华', log='以下是您的请求的解决方案：\n\n```json\n{\n    "reason": "用户直接询问了刘德华的老婆是谁，因此需要通过搜索来获取答案。",\n    "action": "Search",\n    "action_input": "刘德华"\n}\n```\n\n最终答复问题的答案放到这里！'),
   '[\'Andy Lau Tak-wah SBS BBS MH JP, is a Hong Kong actor, singer-songwriter, film producer, and philanthropist. He was named the "Fourth Tiger" among the Five Tiger Generals of TVB in the 1980s as well as one of the Four Heavenly Kings in the 1990s.\', \'Andy Lau (刘德华) type: Hong Kong actor and singer-songwriter.\', \'Andy Lau (刘德华) entity_type: people.\', \'Andy Lau (刘德华) kgmid: /m/01t2h2.\', \'Andy Lau (刘德华) born: September 27, 1961 (age 64 years), Tai Hang, Hong Kong.\', \'Andy Lau (刘德华) spouse: Carol Chu (m. 2008).\', \'Andy Lau (刘德华) upcoming_movie: The Wandering Earth 3.\', \'Andy Lau (刘德华) parents: Lau Lai.\', \'Andy Lau (刘德华) height: 5′ 9″.\', \'刘德华，SBS，MH，JP（英语：Andy Lau Tak Wah；1961年9月27日—），男

添加历史记录

In [21]:
prompt = ChatPromptTemplate.from_messages(
    [
      (
          "system",
          "你是非常强大的助手，你可以使用各种工具来完成人类交给的任务和任何。"
      ),
      MessagesPlaceholder(variable_name="chat_history"),
      ("user",promptTemplate),
      MessagesPlaceholder(variable_name="agent_scratched"),
    ]
)

In [22]:
chat_history=[]

In [26]:
prompt=prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

agent=(
    RunnablePassthrough.assign(
        agent_scratched=lambda x: format_log_to_messages(
            x["input"],
            x["intermediate_steps"],
            template_tool_response=TEMPLATE_TOOL_RESPONSE,
        )
    )
    | prompt
    | llm
    | JSONAgentOutputParser()
)

agent_executor=AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_execution_time=10,
    max_iterations=2
)

In [28]:
chat_history.extend(
    [
        HumanMessage(content="你好，我是老陈"),
        AIMessage(content="你好，老陈，很高兴认识你"),
    ]
)
agent_executor.invoke({"input":"我叫什么","chat_history":chat_history})



> Entering new AgentExecutor chain...
```json
{
    "reason": "根据用户输入，需要获取刘德华的姓名。",
    "action": "Search",
    "action_input": "刘德华"
}
```['Andy Lau Tak-wah SBS BBS MH JP, is a Hong Kong actor, singer-songwriter, film producer, and philanthropist. He was named the "Fourth Tiger" among the Five Tiger Generals of TVB in the 1980s as well as one of the Four Heavenly Kings in the 1990s.', 'Andy Lau (刘德华) type: Hong Kong actor and singer-songwriter.', 'Andy Lau (刘德华) entity_type: people.', 'Andy Lau (刘德华) kgmid: /m/01t2h2.', 'Andy Lau (刘德华) born: September 27, 1961 (age 64 years), Tai Hang, Hong Kong.', 'Andy Lau (刘德华) spouse: Carol Chu (m. 2008).', 'Andy Lau (刘德华) upcoming_movie: The Wandering Earth 3.', 'Andy Lau (刘德华) parents: Lau Lai.', 'Andy Lau (刘德华) height: 5′ 9″.', '刘德华，SBS，MH，JP（英语：Andy Lau Tak Wah；1961年9月27日—），男，汉族，籍贯广东新会，香港男演员、歌手、填词人、监制及出品人，1990年代获封为香港乐坛“四大天王”之一，也是 ...', '刘德华（Andy Lau），1961年9月27日出生于中国香港，华语影视男演员、流行乐歌手、制片人、作词人。1981年，出演个人首部电影《彩云曲》。1985年，发行个人首张音乐专辑《只 ...', '#刘德华为十

{'input': '我叫什么',
 'chat_history': [HumanMessage(content='你好，我是老陈', additional_kwargs={}, response_metadata={}),
  AIMessage(content='你好，老陈，很高兴认识你', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='刘德华的老婆是谁', additional_kwargs={}, response_metadata={}),
  AIMessage(content='刘德华的老婆是朱丽倩', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='你好，我是老陈', additional_kwargs={}, response_metadata={}),
  AIMessage(content='你好，老陈，很高兴认识你', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='你好，我是老陈', additional_kwargs={}, response_metadata={}),
  AIMessage(content='你好，老陈，很高兴认识你', additional_kwargs={}, response_metadata={})],
 'answer': '刘德华'}

In [25]:
chat_history.extend(
    [
        HumanMessage(content="刘德华的老婆是谁"),
        AIMessage(content="刘德华的老婆是朱丽倩"),
    ]
)
agent_executor.invoke({"input":"刘德华的老婆有演过电影吗","chat_history":chat_history})



> Entering new AgentExecutor chain...
```json
{
    "reason": "I have no information about whether Bolei is a movie actor.",
    "action": "Search",
    "action_input": "bolei"
}
```['Bolay restaurants offer healthy dinner and lunch bowls made with fresh ingredients curated by chefs around the country. Come try our fusion of flavors!', 'I am an associate professor in the Computer Science Department at UCLA, where I direct the Vision and Autonomy Intelligence Lab (VAIL). My research is at ...', 'Bolei Zhou. Associate Professor at UCLA. Verified email at ucla.edu - Homepage · Computer VisionRoboticsArtificial Intelligence. ArticlesCited by ...', 'My research lab at UCLA focuses on computer vision, machine learning, robotics, and generative AI. See my webpage at https://boleizhou.github.io/', 'ASSOCIATE PROFESSOR COMPUTER SCIENCE. Engineering VI - Room 295D. Email: bolei@cs.ucla.edu. Phone: (310) 825-7048. Fax: (310) 794-5057. Websites.', 'Bolei Deng is an Assistant Professor at the Gug

{'input': '刘德华的老婆有演过电影吗',
 'chat_history': [HumanMessage(content='你好，我是老陈', additional_kwargs={}, response_metadata={}),
  AIMessage(content='你好，老陈，很高兴认识你', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='刘德华的老婆是谁', additional_kwargs={}, response_metadata={}),
  AIMessage(content='刘德华的老婆是朱丽倩', additional_kwargs={}, response_metadata={})],
 'answer': "No, according to the tool's response, Bolei Zhou has not performed in any movies."}